#Connect to drive

In [53]:
import warnings
import logging
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category = DeprecationWarning)

In [1]:
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
%cd RecSys_Course_AT_PoliMi
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1499, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1499 (delta 113), reused 230 (delta 99), pack-reused 1255
Receiving objects: 100% (1499/1499), 50.98 MiB | 6.32 MiB/s, done.
Resolving deltas: 100% (844/844), done.
/content/RecSys_Course_AT_PoliMi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.0/978.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


#Import Library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from typing import Tuple, Callable, Dict, Optional, List
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from scipy.stats import loguniform
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00


#Load Data

In [4]:
%cd /gdrive/MyDrive/RECSYS
data_train = pd.read_csv('data_train.csv')
data_target_users_test = pd.read_csv('data_target_users_test.csv')
%cd ../../../content/RecSys_Course_AT_PoliMi

/gdrive/MyDrive/RECSYS
/content/RecSys_Course_AT_PoliMi


In [5]:
data_train.columns = ["user_id", "item_id", "Interaction"]

In [6]:
print ("The number of interactions is {}".format(len(data_train)))

The number of interactions is 478730


In [8]:
userID_unique = data_train["user_id"].unique()
itemID_unique = data_train["item_id"].unique()

In [9]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(data_train)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024



#Splitting Data

In [10]:
def preprocess_data(interactions: pd.DataFrame):
    unique_users = interactions.user_id.unique()
    unique_items = interactions.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    interactions = pd.merge(left=interactions,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    interactions = pd.merge(left=interactions,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return interactions


In [11]:
interactions = preprocess_data(data_train)

12638 1 13024
22222 1 22347


In [12]:
interactions

,user_id,item_id,Interaction,mapped_user_id,mapped_item_id
0,1,7,1.0,0,0
1,2,7,1.0,1,0
2,26,7,1.0,24,0
3,36,7,1.0,34,0
4,41,7,1.0,39,0
...,...,...,...,...,...
478725,12962,20368,1.0,12579,22217
478726,12985,21058,1.0,12601,22218
478727,12989,22317,1.0,12605,22219
478728,13009,22339,1.0,12624,22220


In [13]:
def dataset_splits(interactions, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    (user_ids_training, user_ids_test,
     item_ids_training, item_ids_test,
     interactions_training, interactions_test) = train_test_split(interactions.mapped_user_id,
                                                        interactions.mapped_item_id,
                                                        interactions.Interaction,
                                                        test_size=testing_percentage,
                                                        shuffle=True,
                                                        random_state=seed)

    (user_ids_training, user_ids_validation,
     item_ids_training, item_ids_validation,
     interactions_training, interactions_validation) = train_test_split(user_ids_training,
                                                              item_ids_training,
                                                              interactions_training,
                                                              test_size=validation_percentage,
                                                             )

    urm_train = sp.csr_matrix((interactions_training, (user_ids_training, item_ids_training)),
                              shape=(num_users, num_items))

    urm_validation = sp.csr_matrix((interactions_validation, (user_ids_validation, item_ids_validation)),
                              shape=(num_users, num_items))

    urm_test = sp.csr_matrix((interactions_test, (user_ids_test, item_ids_test)),
                              shape=(num_users, num_items))



    return urm_train, urm_validation, urm_test



In [14]:
urm_train, urm_validation, urm_test = dataset_splits(interactions,
                                                     num_users=12638,
                                                     num_items=22222,
                                                     validation_percentage=0.20,
                                                     testing_percentage=0.20)

In [15]:
urm_train

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 306387 stored elements in Compressed Sparse Row format>

In [16]:
urm_validation

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 76597 stored elements in Compressed Sparse Row format>

In [17]:
urm_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>

# Build the Recommendation System

## Import Evaluator

In [292]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation,cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test,cutoff_list=[10])

EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2152 (17.0%) Users that have less than 1 test interactions


In [302]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame()

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

## RP3betaRecommender

In [293]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [298]:
rp3 = RP3betaRecommender(urm_train + urm_validation)
bests = {'topK': 44, 'alpha': 0.21591232212788025, 'beta': 0.15299862700982697}

rp3.fit(**bests)

RP3betaRecommender: URM Detected 230 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 93 ( 0.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4075.15 column/sec. Elapsed time 5.45 sec


In [318]:
result_df, _ = evaluator_test.evaluateRecommender(rp3)
result_df

EvaluatorHoldout: Processed 10486 (100.0%) in 6.09 sec. Users per second: 1722


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.093143,0.162036,0.136137,0.048089,0.083478,0.267838,0.14892,0.110609,0.511158,0.354076,...,0.82972,0.424118,0.82972,0.086468,10.742446,0.998595,0.248248,0.827582,1.378839,0.299739


## P3alpha

In [299]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [300]:
p3alpha = P3alphaRecommender(urm_train + urm_validation)
bests = {'topK': 57, 'alpha': 0.33099429773847305, 'implicit':True, 'normalize_similarity': True}

p3alpha.fit(**bests)

P3alphaRecommender: URM Detected 230 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 93 ( 0.4%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4967.54 column/sec. Elapsed time 4.47 sec


In [317]:
result_df, _ = evaluator_test.evaluateRecommender(p3alpha)
result_df

EvaluatorHoldout: Processed 10486 (100.0%) in 6.44 sec. Users per second: 1627


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.091284,0.156247,0.130494,0.047534,0.082275,0.266254,0.145999,0.107423,0.500572,0.350714,...,0.82972,0.415335,0.82972,0.067953,10.465962,0.998379,0.195093,0.806282,1.334062,0.296473


## Item KNN

In [295]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [297]:
itemknn = ItemKNNCFRecommender(urm_train + urm_validation)
bests = {'similarity': 'tversky',
 'topK': 18,
 'shrink': 10,
 'tversky_alpha': 0.19184100253588082,
 'tversky_beta': 1.6515107809400285}
itemknn.fit(**bests)

ItemKNNCFRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 93 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 6550.03 column/sec. Elapsed time 3.39 sec


In [316]:
result_df, _ = evaluator_test.evaluateRecommender(itemknn)
result_df

EvaluatorHoldout: Processed 10486 (100.0%) in 5.43 sec. Users per second: 1933


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092924,0.160596,0.134392,0.047748,0.081981,0.265426,0.147168,0.109876,0.506962,0.351281,...,0.82972,0.420636,0.82972,0.082021,10.474686,0.99789,0.235481,0.806954,1.632192,0.296453


### Hyperparam Tuning

In [ ]:
def objective_function_ItemKNN_similarities(optuna_trial):

    recommender_instance = ItemKNNCFRecommender(urm_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])

    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 1000),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                  }

    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])


    recommender_instance.fit(**full_hyperp)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
item_optuna_study = optuna.create_study(direction="maximize")

item_save_results = SaveResults()

item_optuna_study.optimize(objective_function_ItemKNN_similarities,
                      callbacks=[item_save_results],
                      n_trials = 50)

## Top popular for the users without info

In [301]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)


    def recommend(self, user_id, at=5):

        recommended_items = self.popular_items[0:at]

        return recommended_items

## Hybrid Models

In [323]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sp.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3

    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*((1-self.alpha)/2) + item_weights_3*((1-self.alpha)/2)

        return item_weights

In [328]:
def objective_function_hybrid(optuna_trial):
  hybridrecommender = ScoresHybridRecommender(urm_train + urm_validation, itemknn, rp3, p3alpha)
  hybridrecommender.fit(alpha = optuna_trial.suggest_float("alpha", 0.01, 1.0, log=True))
  result_df, _ = evaluator_test.evaluateRecommender(hybridrecommender)

  return result_df.loc[10]["MAP"]

In [329]:
hybrid_optuna_study = optuna.create_study(direction="maximize")

hybrid_save_results = SaveResults()

hybrid_optuna_study.optimize(objective_function_hybrid,
                      callbacks=[hybrid_save_results],
                      n_trials = 50)

[I 2023-11-27 19:05:24,133] A new study created in memory with name: no-name-8775cb41-138c-42e9-a02f-d463e505e9f7


ScoresHybridRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 93 ( 0.4%) items with no interactions.
EvaluatorHoldout: Processed 10486 (100.0%) in 7.09 sec. Users per second: 1478


[I 2023-11-27 19:05:31,252] Trial 0 finished with value: 0.0492022318495717 and parameters: {'alpha': 0.24884700700991433}. Best is trial 0 with value: 0.0492022318495717.


ScoresHybridRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 93 ( 0.4%) items with no interactions.
EvaluatorHoldout: Processed 10486 (100.0%) in 5.93 sec. Users per second: 1768


[I 2023-11-27 19:05:37,208] Trial 1 finished with value: 0.04912007014643918 and parameters: {'alpha': 0.06027132518477795}. Best is trial 0 with value: 0.0492022318495717.


...

[I 2023-11-27 19:10:58,545] Trial 47 finished with value: 0.04943440081862799 and parameters: {'alpha': 0.15073916271515536}. Best is trial 25 with value: 0.049493928412487.


ScoresHybridRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 93 ( 0.4%) items with no interactions.
EvaluatorHoldout: Processed 10486 (100.0%) in 7.48 sec. Users per second: 1401


[I 2023-11-27 19:11:06,063] Trial 48 finished with value: 0.04944108773905706 and parameters: {'alpha': 0.10453536176759301}. Best is trial 25 with value: 0.049493928412487.


ScoresHybridRecommender: URM Detected 230 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 93 ( 0.4%) items with no interactions.
EvaluatorHoldout: Processed 10486 (100.0%) in 8.90 sec. Users per second: 1178


[I 2023-11-27 19:11:15,009] Trial 49 finished with value: 0.04902198017613874 and parameters: {'alpha': 0.36818048283031723}. Best is trial 25 with value: 0.049493928412487.


In [319]:
## con itemknn + rp3 = 0.4936
hybrid_optuna_study.best_trial.params

{'alpha': 0.19492256156110832}

In [ ]:
# con itemknn + rp3 + p3 = 0.049493928412487.
alpha = 0.0.18449421068723693

#Prepare Submission

In [274]:
urm_train_validation_test = urm_train + urm_validation + urm_test
urm_train_validation_test

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [289]:
bests ={}

best_recommender = (urm_train_validation_test)
best_recommender.fit(**bests)

RP3betaRecommender: Similarity column 22222 (100.0%), 3882.67 column/sec. Elapsed time 5.72 sec


##Top popular for those users without info

In [285]:
toppop_recommender = TopPopRecommender()
toppop_recommender.fit(urm_train_validation_test)

In [286]:
users_to_recommend = np.array(data_target_users_test["user_id"])
len(users_to_recommend)

10882

## Prepare Sub

In [290]:
def prepare_submission(interactions: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object, toppop_recommender: object):
    users_ids_and_mappings = interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates()
    items_ids_and_mappings = interactions[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(interactions.mapped_item_id, interactions.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        recommendations = recommender.recommend(user_id_array=mapped_user_id, cutoff=10)

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    #assign top popular to the user without interactions
    users_without_info = np.setdiff1d(users_to_recommend,np.array(interactions[interactions.user_id.isin(users_to_recommend)][['user_id', 'mapped_user_id']].drop_duplicates().user_id))
    for user_id in users_without_info:
      recommendations = toppop_recommender.recommend(user_id, at = 10)
      submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    return submission

In [ ]:
submission = prepare_submission(interactions, users_to_recommend, urm_train_validation_test, best_recommender, toppop_recommender)

In [ ]:
submission

In [ ]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
%cd /gdrive/MyDrive/RECSYS
write_submission(submission)

/gdrive/MyDrive/RECSYS
